<a href="https://colab.research.google.com/github/simi12345/Sentiment-Analysis/blob/main/Copy_of_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape

In [ ]:
!pip install nlp-id

     |████████████████████████████████| 7.5MB 8.3MB/s 
     |████████████████████████████████| 7.0MB 15.5MB/s 
     |████████████████████████████████| 1.5MB 53.7MB/s 
  Created wheel for nlp-id: filename=nlp_id-0.1.10.0-cp37-none-any.whl size=7723175 sha256=f4c667fa12e8a688e16340dfc6e0efc6a3792dbf2e6fcee5ad34184f8e0ad8ae
  Stored in directory: /root/.cache/pip/wheels/a5/b6/2b/146e3f2a36d470240782b5b54021d994c974aaefeee5561e77
  Created wheel for nltk: filename=nltk-3.4.5-cp37-none-any.whl size=1449907 sha256=551e2d91b33aeb4e3e2c86795d43a898f2662fdfc4debf84ff2bcbbfadbb5c37
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=0c934d177ac45b4cdae7b1f0f60bbbd9bcdb081e69868b8d519235d86bf260b5
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built nlp-id nltk wget
  Found existing installati

In [ ]:
!pip install emoji

     |████████████████████████████████| 133kB 9.0MB/s 


In [ ]:
!pip install PySastrawi

     |████████████████████████████████| 215kB 9.0MB/s 


In [ ]:
import nltk
import random
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import csv
from nltk.corpus import stopwords
import string
import emoji
import pandas as pd


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn import  model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
with open('dataset/dataset_tweet_sentimen_tayangan_tv.csv', 'r',
          encoding="utf8") as nodecsv: # Buka file
    csvreader = csv.reader(nodecsv) # membaca data
    # Menyusun data dalam list dan menghilangkan header data
    csv = [n for n in csvreader][1:]

In [ ]:
print(csv[10:])

In [ ]:
data = pd.read_csv ('dataset/dataset_tweet_sentimen_tayangan_tv.csv', encoding='latin-1')

data.head(5)

,ï»¿Id,Sentiment,Acara TV,Jumlah Retweet,Text Tweet
0,1,positive,HitamPutihTransTV,12,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S..."
1,2,positive,HitamPutihTransTV,6,Selamat berbuka puasa Semoga amal ibadah hari ...
2,3,positive,HitamPutihTransTV,9,"Ada nih di trans7 hitam putih, dia dpt penghar..."
3,4,positive,HitamPutihTransTV,2,selamat ya mas @adietaufan masuk hitamputih
4,5,positive,HitamPutihTransTV,1,Asiknya nonton Hitam Putih Trans7


In [ ]:
data.Sentiment.value_counts()

negative    200
positive    200
Name: Sentiment, dtype: int64

In [ ]:
df= data [['Text Tweet','Sentiment']]
df.head()

,Text Tweet,Sentiment
0,"Undang @N_ShaniJKT48 ke hitamputih, pemenang S...",positive
1,Selamat berbuka puasa Semoga amal ibadah hari ...,positive
2,"Ada nih di trans7 hitam putih, dia dpt penghar...",positive
3,selamat ya mas @adietaufan masuk hitamputih,positive
4,Asiknya nonton Hitam Putih Trans7,positive


In [ ]:
emoticons_str = r"""
  (?:
      [:=;] # Eyes
      [oO\-]? # Nose (optional) [D\)\]\(\]/\\OpP] # Mouth
  )"""
regex_str = []
regex_str.append(emoticons_str)
regex_str.append(r'<[^>]+>')# HTML tags
regex_str.append(r'(?:@[\w_]+)')# @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [ ]:
nltk.download('punkt')
from nlp_id.stopword import StopWord
stopword = StopWord()

punctuation = list(string.punctuation)
stop = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nlp_id.stopword import StopWord #stopword B.indo
stopword = StopWord()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def tokenize(s):
  tokens = tokens_re.findall(s)
  return tokens

def cleanTweet(token,regex):
    terms_all = [emoji.demojize(term) for term in token if term.lower() not in stop and not regex.match(term)]
    return terms_all

In [ ]:
import operator

exclude_str = []
exclude_str.append(emoticons_str)
exclude_str.append(r'<[^>]+>')# HTML tags
exclude_str.append(r'(?:@[\w_]+)')# @-mentions
exclude_str.append(r'(?:&[\w_]+)')
exclude_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
exclude_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
exclude_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers

exclude_re = re.compile(r'('+'|'.join(exclude_str)+')', re.VERBOSE | re.IGNORECASE)

In [ ]:
def stemming (text):
  return stemmer.stem(text)

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
data['Text Tweet'] = data ['Text Tweet'].apply(stemming)

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['Text Tweet'],df['Sentiment'],test_size=0.2)

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df['Text Tweet'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
Train_X_Tfidf.shape

(320, 1755)

In [ ]:
Train_Y.shape

(320,)

In [ ]:
import nltk.classify
from sklearn.svm import LinearSVC

In [ ]:
SVM = SVC()
SVM.fit(Train_X_Tfidf,Train_Y)

predictions_SVM = SVM.predict(Test_X_Tfidf) #preditc pada data test

print("SVM Accuracy Score --> ", accuracy_score(predictions_SVM, Test_Y)*100 )
print("SVM Recall Score --> ", recall_score(predictions_SVM, Test_Y)*100 )
print("SVM Precision Score --> ", precision_score(predictions_SVM, Test_Y)*100 )
print("SVM f1 Score --> ", f1_score(predictions_SVM, Test_Y)*100 )

SVM Accuracy Score -->  87.5
SVM Recall Score -->  88.88888888888889
SVM Precision Score -->  84.21052631578947
SVM f1 Score -->  86.48648648648648


In [ ]:
import sklearn.metrics as metrics

In [ ]:
lr_model=LogisticRegression(C=0.7,random_state=42)
lr_model.fit(Train_X_Tfidf,Train_Y.ravel())
lr_predict_test = lr_model.predict(Test_X_Tfidf)

# training metrics
print("Accuracy:{0:.4f}".format(metrics.accuracy_score(Test_Y,lr_predict_test)))
print()

#print(metrics.confusion_matrix(Test_Y,lr_predict_test))
#print()
print("Classification Report")
print(metrics.classification_report(Test_Y,lr_predict_test))

Accuracy:0.8750

Classification Report
              precision    recall  f1-score   support

           0       0.92      0.83      0.88        42
           1       0.83      0.92      0.88        38

    accuracy                           0.88        80
   macro avg       0.88      0.88      0.88        80
weighted avg       0.88      0.88      0.88        80



In [ ]:
punctuation = list(string.punctuation)
stop = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“']

In [ ]:
tokens = []
for c in csv:
  tokens.append(tokenize(c[4]))

print(tokens[:5])

[['Undang', '@N_ShaniJKT48', 'ke', 'hitamputih', ',', 'pemenang', 'SSK', 'JKT', '48', 'harusnya', 'mJKT', '48', 'ini', 'lebih', 'Layak', 'di', 'Undang', 'karena', 'prestasinya'], ['Selamat', 'berbuka', 'puasa', 'Semoga', 'amal', 'ibadah', 'hari', 'ni', 'diterima', 'Allah', '#hitamputih'], ['Ada', 'nih', 'di', 'trans', '7', 'hitam', 'putih', ',', 'dia', 'dpt', 'penghargaan', 'juga', 'di', 'norwegia', '#hitamputih'], ['selamat', 'ya', 'mas', '@adietaufan', 'masuk', 'hitamputih'], ['Asiknya', 'nonton', 'Hitam', 'Putih', 'Trans', '7']]


In [ ]:
all_word=[]
from collections import Counter
count_all = Counter()
for token in tokens:
  cleanToken = cleanTweet(token,exclude_re)
  terms_all = [term.lower() for term in cleanToken]
  count_all.update(terms_all)
  for w in terms_all:
      all_word.append(w)
print(count_all.most_common(100))

[('mata', 61), ('najwa', 60), ('hitam', 40), ('putih', 39), ('acara', 32), ('nonton', 29), ('keren', 29), ('orang', 29), ('ga', 28), ('tv', 25), ('indonesia', 21), ('ahok', 19), ('trans', 18), ('metro', 18), ('malam', 15), ('tdk', 15), ('anak', 14), ('ilc', 14), ('semoga', 13), ('menginspirasi', 13), ('andy', 13), ('seru', 11), ('suka', 10), ('liat', 10), ('miris', 10), ('tp', 10), ('gak', 10), ('tvone', 10), ('salut', 10), ('kick', 10), ('nih', 9), ('kalo', 9), ('org', 9), ('bikin', 8), ('hukum', 8), ('dg', 8), ('bapak', 8), ('debat', 8), ('anis', 8), ('undang', 7), ('hati', 7), ('kasih', 7), ('lihat', 7), ('pake', 7), ('hidup', 7), ('tak', 7), ('ngomong', 7), ('lawyers', 7), ('club', 7), ('selamat', 6), ('terima', 6), ('utk', 6), ('setuju', 6), ('lg', 6), ('dgn', 6), ('sbg', 6), ('klo', 6), ('memalukan', 6), ('mati', 6), ('bs', 6), ('kalah', 6), ('krn', 6), ('sakit', 6), ('anies', 6), ('hitamputih', 5), ('masuk', 5), ('menarik', 5), ('inspiratif', 5), ('cinta', 5), ('tamu', 5), ('kay

In [ ]:
len(all_word)

2660

In [ ]:
#inisialisasi lematizer dan postagger
from nlp_id.postag import PosTag
postagger = PosTag()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid resu

In [ ]:
from nlp_id.lemmatizer import Lemmatizer
lemmatizer = Lemmatizer() #mendapatkan kata dasar

In [ ]:
all_stmword = [lemmatizer.lemmatize(w) for w in all_word]

In [ ]:
print(all_stmword[:20])

['undang', 'hitamputih', 'menang', 'ssk', 'jkt', 'mjkt', 'layak', 'undang', 'prestasi', 'selamat', 'buka', 'puasa', 'moga', 'amal', 'ibadah', 'ni', 'terima', 'allah', 'nih', 'trans']


In [ ]:
pos = [postagger.get_pos_tag(w)[0] for w in all_stmword] #memberi label

In [ ]:
print(pos[:10])

[('undang', 'NN'), ('hitamputih', 'VB'), ('menang', 'VB'), ('ssk', 'NN'), ('jkt', 'NN'), ('mjkt', 'NN'), ('layak', 'JJ'), ('undang', 'NN'), ('prestasi', 'NN'), ('selamat', 'JJ')]


In [ ]:
allowed_word_types = ["ADV","JJ","ADJP","VB"]
allowed_words=[w[0] for w in pos if w[1] in allowed_word_types]

In [ ]:
print(allowed_words[:20])

['hitamputih', 'menang', 'layak', 'selamat', 'buka', 'ni', 'terima', 'hitam', 'putih', 'selamat', 'masuk', 'hitamputih', 'nonton', 'hitam', 'putih', 'hitam', 'putih', 'hitam', 'putih', 'hitam']


In [ ]:
#Menyusun distribusi kata berdasarkan kemunculannya
allwords = nltk.FreqDist(allowed_words)

#Mengambil 5000 pertama dari distribusi kata berdasarkan frekwensi kemunculannya
tuple_features = list(allwords.most_common())[:500]
word_features = [k for (k,v) in tuple_features]
#Memeriksa word_features
print(word_features[:100])

['hitam', 'putih', 'nonton', 'seru', 'hidup', 'tp', 'terima', 'bikin', 'rusak', 'kalah', 'lihat', 'tunggu', 'ngomong', 'bilang', 'coba', 'selamat', 'mati', 'pimpin', 'pilih', 'bicara', 'jaga', 'hitamputih', 'masuk', 'bagus', 'tayang', 'hebat', 'dengar', 'salah', 'langsung', 'mantap', 'bangga', 'dukung', 'pikir', 'tua', 'metrotv', 'lawan', 'buka', 'ni', 'pribadi', 'beda', 'kaya', 'mimpi', 'muncul', 'butuh', 'tindak', 'cerdas', 'lucu', 'besar', 'pulang', 'bawa', 'ubah', 'miskin', 'menang', 'bantu', 'baik', 'bahas', 'jujur', 'ikut', 'takut', 'sesuai', 'lupa', 'bayar', 'kagum', 'sejuk', 'milik', 'kuat', 'jadi', 'berani', 'dki', 'uji', 'malas', 'didik', 'tai', 'lancar', 'nyanyi', 'hibur', 'ustadz', 'asli', 'ucap', 'berat', 'aneh', 'sedih', 'sulit', 'parah', 'muda', 'hendak', 'paksa', 'trus', 'ketawa', 'pergi', 'mbak', 'toleransi', 'buruk', 'medsos', 'dijalanin', 'dipikirin', 'taksi', 'pasrah', 'darurat', 'kucing']


In [ ]:
document=[]
for w in csv:

  document.append((w[4],w[1])) #w4 full text, w1 sentimen

In [ ]:
def find_features(document):
  words = word_tokenize(document)
  features = {}
  for w in word_features:
    features[w] = (w in words)
  return features

In [ ]:
nltk.download('punkt')
featuresets = [(find_features(rev), category) for (rev, category) in document]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(featuresets[:5])

[({'hitam': False, 'putih': False, 'nonton': False, 'seru': False, 'hidup': False, 'tp': False, 'terima': False, 'bikin': False, 'rusak': False, 'kalah': False, 'lihat': False, 'tunggu': False, 'ngomong': False, 'bilang': False, 'coba': False, 'selamat': False, 'mati': False, 'pimpin': False, 'pilih': False, 'bicara': False, 'jaga': False, 'hitamputih': True, 'masuk': False, 'bagus': False, 'tayang': False, 'hebat': False, 'dengar': False, 'salah': False, 'langsung': False, 'mantap': False, 'bangga': False, 'dukung': False, 'pikir': False, 'tua': False, 'metrotv': False, 'lawan': False, 'buka': False, 'ni': False, 'pribadi': False, 'beda': False, 'kaya': False, 'mimpi': False, 'muncul': False, 'butuh': False, 'tindak': False, 'cerdas': False, 'lucu': False, 'besar': False, 'pulang': False, 'bawa': False, 'ubah': False, 'miskin': False, 'menang': False, 'bantu': False, 'baik': False, 'bahas': False, 'jujur': False, 'ikut': False, 'takut': False, 'sesuai': False, 'lupa': False, 'bayar': 

In [ ]:
# menyusun data training dan testing menguji model
trainsize = round(len(featuresets)*0.7)

random.shuffle(featuresets)
training_set = featuresets[:trainsize]
testing_set = featuresets[trainsize:]

In [ ]:
print(testing_set[:5])

[({'hitam': False, 'putih': False, 'nonton': False, 'seru': False, 'hidup': False, 'tp': False, 'terima': False, 'bikin': False, 'rusak': False, 'kalah': False, 'lihat': False, 'tunggu': False, 'ngomong': False, 'bilang': False, 'coba': False, 'selamat': False, 'mati': False, 'pimpin': False, 'pilih': False, 'bicara': False, 'jaga': False, 'hitamputih': False, 'masuk': False, 'bagus': False, 'tayang': False, 'hebat': False, 'dengar': False, 'salah': False, 'langsung': False, 'mantap': False, 'bangga': False, 'dukung': False, 'pikir': False, 'tua': False, 'metrotv': False, 'lawan': False, 'buka': False, 'ni': False, 'pribadi': False, 'beda': False, 'kaya': False, 'mimpi': False, 'muncul': False, 'butuh': False, 'tindak': False, 'cerdas': False, 'lucu': False, 'besar': False, 'pulang': False, 'bawa': False, 'ubah': False, 'miskin': False, 'menang': False, 'bantu': False, 'baik': False, 'bahas': False, 'jujur': False, 'ikut': False, 'takut': False, 'sesuai': False, 'lupa': False, 'bayar':

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(25)

Classifier accuracy percent: 59.166666666666664
Most Informative Features
                   hitam = True           positi : negati =     11.3 : 1.0
                   putih = True           positi : negati =      6.4 : 1.0
              hitamputih = True           positi : negati =      2.4 : 1.0
                   jelas = True           negati : positi =      2.3 : 1.0
                    baik = True           negati : positi =      2.3 : 1.0
                   salah = True           negati : positi =      2.3 : 1.0
                 ngomong = True           negati : positi =      2.3 : 1.0
                  nonton = True           positi : negati =      1.8 : 1.0
                   metro = True           negati : positi =      1.7 : 1.0
                   hidup = True           positi : negati =      1.7 : 1.0
                  tayang = True           positi : negati =      1.7 : 1.0
                   tegas = True           positi : negati =      1.7 : 1.0
                   mampu =

In [ ]:
import nltk.classify
from sklearn.svm import LinearSVC

In [ ]:
from nltk.metrics.scores import (precision, recall, f_measure)
import collections

refsets = collections.defaultdict(set)
testsets = collections.defaultdict(set)

for i, (feats, label) in enumerate(testing_set):
  refsets[label].add(i)
  observed = classifier.classify(feats)
  testsets[observed].add(i)


print('Recall:', recall(refsets['positive'], testsets['positive'])) #jumlah positive yg berhasil ditebak
print('Precision:', precision(refsets['positive'], testsets['positive'])) #brp tebakan positve yg benar dari seluruh tebakan positive
print('f1:', f_measure(refsets['positive'], testsets['positive'],alpha=0.5))

Recall: 0.8870967741935484
Precision: 0.5670103092783505
f1: 0.6918238993710693


In [ ]:
#menyimpan model yang telah ditrainning ke file sav
filename = 'nb_model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [ ]:
#load model classifier nb dari file
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#melakukan klasifikasi menggunakan model classifier nb yang telah diload
lsPrediksi = []
for (rev, category) in document[trainsize:]:
  result=loaded_model.classify(find_features(rev))
  lsPrediksi.append([rev, result, category])

In [ ]:
print(lsPrediksi[:10])

[['Orang Belanda aja peduli sama orang2 indonesia yg berada d daerah terisolir, gimana dg pemerintah indonesia? Zhonk kickandyMetroTV', 'negative'], ['Merinding deh nonton #KickAndyMetroTV', 'positive'], ['Indonesia2. Bahkan kita dididik untuk menjadi rusak.. #kickandymetrotv', 'positive'], ['Parahnya kualitas pendidikan di Indonesia  #kickandymetrotv', 'positive'], ['demi apapun ya kick andy sekarang botak?  #kickandymetrotv', 'positive'], ['jadi merasa malu krn kalah dari org yg tdk sempurna tapi mempunyai yg lebih dari org sempurna #kickandymetrotv', 'negative'], ['Hukum di Indonesia masih jauh dari kata ADIL #KickAndyMetroTV', 'positive'], ['aku kira makin tua makin keladi, malah jd macan ompong. Melempem. kecewa liat amin rais di @kickandymetrotv', 'negative'], ['#kickAndyMetroTV MEDIA harus mengisi lebih banyak ttg kisah sukses di TV, bukan gosip, berita rusuh & acara2 GG JELAS, tai kucing!! #CATET', 'negative'], ['#kickAndyMetroTV jika ingin dihargai orang, hargailah orang lain 